In [ ]:
from requests_html import HTMLSession
import html, json
from twilio.rest import Client

In [2]:
url = 'https://www.costco.com/kirkland-signature-whole-wheel-parmigiano-reggiano%2c-72-lbs..product.100096211.html'
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.32 (KHTML, like Gecko) Chrome/78.0.6230 Safari/537.32"}

# Your Account Sid and Auth Token from twilio.com/console
account_sid = 'AC...'
auth_token = '...'
twilio_from_number = '+1585475....'
sms_destination = '+1585475....'

client = Client(account_sid, auth_token)

In [4]:
def getRawHtml(url, headers):
    session = HTMLSession()
    r = session.get(url, headers=headers)
    
    '''
    Costco returns everything in the initial HTML page load, they don't have an API.
    Therefore all the info we need is in there, unfortunately as JS objects in one of the
    many <script> elements. Here we find the <script> element we care about.
    '''
    element = r.html.find('#productImageLibrary + script')[0].html
    element = html.unescape(element)
    return element

In [5]:
def parseHtml(element):
    '''
    Absent of a nice way to have Python execute JS, which would allow us to 
    access the object we care about, we string parse the <script> contents
    until we have something that's good enough for json.loads() to work with,
    at which point we can pretend it's a regular Python object.
    '''
    opening_tag = element.index('>') + 1
    closing_tag = len(element) - element[::-1].index('<') - 1
    raw_js = element[opening_tag:closing_tag]

    products_start = raw_js.index('products')
    products = raw_js[products_start:]

    json_start = products.index('[')

    raw_json = products[json_start:]
    clean_json = raw_json.replace('\r', '').replace('\n', '').replace('\t', '')

    # avert your eyes
    products_end = len(clean_json) - clean_json[::-1].index(';') - 1
    clean_json = clean_json[:products_end]
    products_end = len(clean_json) - clean_json[::-1].index(';') - 1
    clean_json = clean_json[:products_end]

    # json.loads() is incredibly picky with spaces
    clean_json = clean_json.replace('  ', ' ')
    
    return clean_json

In [6]:
def getInventory():
    raw_html = getRawHtml(url, headers)
    json_string = parseHtml(raw_html)
    
    products = json.loads(json_string)

    inventory = products[0][0]['inventory']

    if inventory:
        return True
    else:
        return False

In [7]:
def sendSMS(msg):
    message = client.messages \
                .create(
                     body=msg,
                     from_=twilio_from_number,
                     to=sms_destination
                 )

In [ ]:
if getInventory():
    sendSMS('Big Cheese in stock at Costco ' + url)

## For running in GCP Cloud Function/Lambda

Note that Akamai denies GCP IPs with prejudice :) so this won't work without an Akamai bypass.

Create an HTTP triggered Cloud Function with source:
```python
from requests_html import HTMLSession
import html, json, os
from twilio.rest import Client

url = 'https://www.costco.com/kirkland-signature-whole-wheel-parmigiano-reggiano%2c-72-lbs..product.100096211.html'
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.32 (KHTML, like Gecko) Chrome/78.0.6230 Safari/537.32"}

account_sid = os.environ['TW_ACC']
auth_token = os.environ['TW_AUTH']
twilio_from_number = os.environ['FROM']
sms_destination = os.environ['TO']

client = Client(account_sid, auth_token)

def getRawHtml(url, headers):
    session = HTMLSession()
    r = session.get(url, headers=headers)
    element = r.html.find('#productImageLibrary + script')[0].html
    element = html.unescape(element)
    return element

def parseHtml(element):
    opening_tag = element.index('>') + 1
    closing_tag = len(element) - element[::-1].index('<') - 1
    raw_js = element[opening_tag:closing_tag]

    products_start = raw_js.index('products')
    products = raw_js[products_start:]

    json_start = products.index('[')

    raw_json = products[json_start:]
    clean_json = raw_json.replace('\r', '').replace('\n', '').replace('\t', '')

    products_end = len(clean_json) - clean_json[::-1].index(';') - 1
    clean_json = clean_json[:products_end]
    products_end = len(clean_json) - clean_json[::-1].index(';') - 1
    clean_json = clean_json[:products_end]

    clean_json = clean_json.replace('  ', ' ')
    
    return clean_json
    
def getInventory():
    raw_html = getRawHtml(url, headers)
    json_string = parseHtml(raw_html)
    
    products = json.loads(json_string)

    inventory = products[0][0]['inventory']

    if inventory:
        return True
    else:
        return False
        
def sendSMS(msg):
    message = client.messages \
                .create(
                     body=msg,
                     from_=twilio_from_number,
                     to=sms_destination
                 )

def checkStock(request):
    if getInventory():
        sendSMS('Big Cheese in stock at Costco ' + url)
        return 'In stock'
    else:
        return 'Out of stock'
```

And requirements.txt:
```
# Function dependencies, for example:
# package>=version
twilio==6.25.2
requests-html==0.10.0
```

Lang: Python 3.7
Memory: 128 MB
Function: checkStock
Max invocations: 1

Set env vars TW_ACC, TW_AUTH, FROM, and TO.

Create a Cloud Scheduler, with frequency `* */2 * * *` (every 2 hours) making an HTTP GET to the Cloud Function trigger. Or use https://crontab.guru to create other timings.